In [2]:
# Imports

import steamdata
import preprocess
import pandas as pd
import database
import preprocess
from nltk.stem.wordnet import WordNetLemmatizer

#### **1) DATA ACQUISITION**

In [ ]:
# Gets top 100 -> DO NOT RUN

app_ids = steamdata.get_app_ids() # -> Retrieves best 100 of top sellers weekly chart.
steamdata.update_data() # -> Sends an API Call for each game, parses the response and saves in DB

#### **2) PRE-PROCESSING**

In [3]:
# Gets all data from database
df = database.get_data_from_database()
df

,app_id,recommendationid,review,voted_up,votes_up,votes_funny,weighted_vote_score
0,730,146314267,this game is amazing\ni got made fun of for be...,1,3965.0,2281.0,0.933550655841827393
1,730,147244655,167 Hard-working achievements gone… How dare y...,0,573.0,50.0,0.895330309867858887
2,730,147189280,"Valve did an Activision-Blizzard, and force up...",0,460.0,2.0,0.889447510242462158
3,730,147147909,"Final Review for the CSGO , end of an era!!!",1,1002.0,42.0,0.880676031112670898
4,730,145932685,"Good game, ruined by cheaters",1,850.0,48.0,0.87881171703338623
...,...,...,...,...,...,...,...
67393,1273400,135318201,fun,1,0.0,0.0,0
67394,1273400,135305444,ד,1,0.0,0.0,0
67395,1273400,135293026,"The game is what it promises, above all.If you...",1,0.0,0.0,0
67396,1273400,135292130,I love coming home from work just to go back t...,1,0.0,0.0,0


In [4]:
import inflect
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
import num2words
from nltk.stem.wordnet import WordNetLemmatizer
import emoji
import contractions
from spellchecker import SpellChecker

# Pre-processing

p = inflect.engine()
stop_words = set(stopwords.words('english'))
spell = SpellChecker()
lemmatizer = WordNetLemmatizer()

index = 0

clean_col = []
for word in df["review"]:

    # contractions 
    clean_word = contractions.fix(word)

    # emojis
    clean_word = emoji.demojize(clean_word)

    # lowercase
    clean_word = clean_word.lower()

    # Special characters cleaning
    clean_word = re.sub(r'[^\w\s]','', clean_word)
    clean_word = re.sub('_', '', clean_word)

    # Additional spaces cleaning
    clean_word = clean_word.strip()
    clean_word = re.sub(' {2,}', ' ', clean_word)

    # Tokenization -> Word by word processing
    words = word_tokenize(clean_word)

    clean_words = []
    for word in words:
        # Convert numbers to words
        if word.isdigit():
            word = num2words.num2words(word)

        # Lemmatize
        word = lemmatizer.lemmatize(word)

        # Spell Check
        if spell.unknown(word):
            word = spell.correction(word)

        # Clean stop_words
        if (word not in stop_words):
            clean_words.append(word)

    clean_col.append(clean_words)
    index = index + 1
    print(index)
df["review_clean"] = clean_col

['game', 'amazing', 'got', 'made', 'fun', 'jew', 'even', 'jewish']
1
[None, 'hardworking', 'achievement', 'gone', 'dare']
2
['valve', None, 'force', 'upgraded', 'everyone', 'playing', 'play', 'counter', 'strike', 'two', 'valve', 'sequel', 'major', 'patch', 'top', 'replaced', 'store', 'page', 'million', 'review', 'stats', 'attributed', 'csi', 'even', 'though', 'earned', 'underhanded', 'disgusting', 'behaviour', 'expect', 'better', 'valve', 'company', 'support']
3
['final', 'review', 'end', 'era']
4
['good', 'game', 'ruined', 'cheater']
5
['pa', 'potato', 'run']
6
['brought', 'ten', 'year', 'ago', 'ha', None, 'csi', 'ikea', 'never', 'came', 'house', 'wa', 'away', 'replace', 'black', 'chair', 'yellow', 'armchair', 'thought', 'wa', 'better', 'update', 'different', 'product', 'put', 'purchase', 'back', 'library']
7
['removed', 'merged', 'review', 'dropped', 'support', 'mac', 'cool', 'fair', 'scummy']
8
['took', 'away', 'mac', 'support', 'removing', 'game', 'doe', 'support', 'update', 'inter

: 